# Performance

In this notebook we shall discuss the numerical performance of Kingdon.

In [1]:
import timeit
import numpy as np
from math import comb
import random

from kingdon import Algebra

In [2]:
alg = Algebra(3, 0, 1)

We shall investigate the commutator product of a bivector with a vector; a common and important computation since this is the infinitesimal transformation of the vector under the transformation generated by the bivector.

We make a simple function that spits out an array of random values for the vector and bivector. (We make this into a function such that we can reuse it later.)

In [3]:
def generate_data(num_rows):
    bivector_shape = (comb(alg.d, 2), num_rows) if num_rows != 1 else comb(alg.d, 2)
    vector_shape = (comb(alg.d, 1), num_rows) if num_rows != 1 else comb(alg.d, 1)

    bvals = np.random.random(bivector_shape)
    uvals = np.random.random(vector_shape)
    return bvals, uvals

Let us first compute the commutator `b.cp(u)` for a 1-dimensional array.

In [4]:
bvals, uvals = generate_data(1)
b = alg.bivector(bvals)
u = alg.vector(uvals)

The first call will be expensive to make, because kingdon will have to generate the optimal code for this computation.

In [5]:
timeit.timeit('b.cp(u)', number=1, globals=globals())

0.7187370999963605

However, subsequent calls are significantly faster:

In [6]:
number = 10**5
t = timeit.timeit('b.cp(u)', number=number, globals=globals())
periter_python = t/number
print(f'Total time: {t}, time per iteration: {periter_python:.2E}')

Total time: 0.34497990000090795, time per iteration: 3.45E-06


The computation will of course slow down once we go for a two-dimensional arrays:

In [7]:
# bvals, uvals = generate_data(10000)
# b = alg.bivector(bvals)
# u = alg.vector(uvals)

In [8]:
# number = 10**4
# t = timeit.timeit('b.cp(u)', number=number, globals=globals())
# print(f'Total time: {t}, time per iteration: {t/number:.2E}')

Now let us do both of these scenarios again, but with numba enabled:

In [9]:
alg = Algebra(3, 0, 1, numba=True)

In [10]:
bvals, uvals = generate_data(1)
b = alg.bivector(bvals)
u = alg.vector(uvals)

In [11]:
timeit.timeit('b.cp(u)', number=1, globals=globals())

0.3909745999990264

In [12]:
number = 10**5
t = timeit.timeit('b.cp(u)', number=number, globals=globals())
periter_numba = t/number
print(f'Total time: {t}, time per iteration: {periter_numba:.2E}')

Total time: 0.2105652000027476, time per iteration: 2.11E-06


In [13]:
# bvals, uvals = generate_data(10000)
# b = alg.bivector(bvals)
# u = alg.vector(uvals)

In [14]:
# number = 10**4
# t = timeit.timeit('b.cp(u)', number=number, globals=globals())
# print(f'Total time: {t}, time per iteration: {t/number:.2E}')

The difference between numba and pure python becomes smaller as the size of the arrays increaces, because that means we spend more time in numpy and thus the lesser performance of Python becomes less relevant.

In [15]:
def generate_data(num_rows):
    bivector_shape = (2**alg.d, num_rows) if num_rows != 1 else 2**alg.d
    vector_shape = (2**alg.d, num_rows) if num_rows != 1 else 2**alg.d

    bvals = np.random.random(bivector_shape)
    uvals = np.random.random(vector_shape)
    return bvals, uvals

In [16]:
alg = Algebra(3, 0, 1)

In [17]:
xvals, yvals = generate_data(1)
x = alg.multivector(xvals)
y = alg.multivector(yvals)

In [18]:
timeit.timeit('x.cp(y)', number=1, globals=globals())

1.377215000000433

In [19]:
number = 10**5
t = timeit.timeit('x.cp(y)', number=number, globals=globals())
periter_python_full = t/number
print(f'Total time: {t}, time per iteration: {periter_python_full:.2E}')

Total time: 1.1238376000037533, time per iteration: 1.12E-05


In [20]:
periter_python_full/periter_python

3.2576900857145463

In [21]:
alg = Algebra(3, 0, 1, numba=True)

In [22]:
xvals, yvals = generate_data(1)
x = alg.multivector(xvals)
y = alg.multivector(yvals)

In [23]:
timeit.timeit('x.cp(y)', number=1, globals=globals())

1.6741316999978153

In [24]:
number = 10**5
t = timeit.timeit('x.cp(y)', number=number, globals=globals())
periter_numba_full = t/number
print(f'Total time: {t}, time per iteration: {periter_python_full:.2E}')

Total time: 0.3859777999969083, time per iteration: 1.12E-05


In [25]:
periter_numba_full/periter_numba

1.83305598452105